In [ ]:
!pip install autogluon
!pip install -U ipywidgets

In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')
test.head()

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
train=train.drop(['id'],axis=1)
train.head()

In [ ]:
test=test.drop(['id'],axis=1)
test.head()

In [ ]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9]+', '_', x))
train.head()

In [ ]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9]+', '_', x))
test.head()

In [ ]:
train.isnull().sum()

In [ ]:
round(train.isnull().sum()*100/len(train),2)

In [ ]:
train[train.isna().any(axis=1)].head()

In [ ]:
train.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

In [ ]:
counter=0
for i in test.select_dtypes(include=['object']).columns.tolist():
    if (len(list(set(train[i].unique().tolist())^set(test[i].unique().tolist())))!=0):
        print(i ,'need to be worked on')
        counter+=1
    else:
        continue
if(counter==0):
    print('No work needed')

In [ ]:
train[test.select_dtypes(include=['object']).columns.tolist()].head()

In [ ]:
Brand=train['Brand'].value_counts().to_frame()
Brand.reset_index()

In [ ]:
Material=train['Material'].value_counts().to_frame()
Material.reset_index()

In [ ]:
Size=train['Size'].value_counts().to_frame()
Size.reset_index()

In [ ]:
Laptop_Compartment=train['Laptop_Compartment'].value_counts().to_frame()
Laptop_Compartment.reset_index()

In [ ]:
Waterproof=train['Waterproof'].value_counts().to_frame()
Waterproof.reset_index()

In [ ]:
Style=train['Style'].value_counts().to_frame()
Style.reset_index()

In [ ]:
Color=train['Color'].value_counts().to_frame()
Color.reset_index()			

In [ ]:
train.duplicated().value_counts()

In [ ]:
label = 'Price'
train[label].describe()

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
predictor = TabularPredictor(label=label,eval_metric ='rmse',
                            problem_type="regression").fit( train,
                                                            time_limit=3600*1,verbosity=3,
                                                            presets='best_quality',  # Ensures deep learning models are used
                                                            ag_args_fit={'num_gpus': 1})  # Forces GPU usage
results = predictor.fit_summary()

In [ ]:
predictor.leaderboard()

In [ ]:
df = predictor.predict(test).to_frame(name=label)
df.head()

In [ ]:
sol=pd.read_csv('/kaggle/input/playground-series-s5e2/sample_submission.csv')
sol.head()

In [ ]:
sol['Price']=df[label]
sol.head()

In [ ]:
sol.to_csv('1-autog.csv',index=False)